In [86]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud" id="h5o-5"><h2>--- Day 11: Radioisotope Thermoelectric Generators ---</h2><p>You come upon a column of four floors that have been entirely sealed off from the rest of the building except for a small dedicated lobby.  There are some radiation warnings and a big sign which reads "Radioisotope Testing Facility".</p>
<p>According to the project status board, this facility is currently being used to experiment with <a href="https://en.wikipedia.org/wiki/Radioisotope_thermoelectric_generator">Radioisotope Thermoelectric Generators</a> (RTGs, or simply "generators") that are designed to be paired with specially-constructed microchips. Basically, an RTG is a highly radioactive rock that generates electricity through heat.</p>
<p>The <span title="The previous version, model number PB-NUK, used Blutonium.">experimental RTGs</span> have poor radiation containment, so they're dangerously radioactive. The chips are prototypes and don't have normal radiation shielding, but they do have the ability to <em>generate an electromagnetic radiation shield when powered</em>.  Unfortunately, they can <em>only</em> be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.</p>
<p>In other words, if a chip is ever left in the same area as another RTG, and it's not connected to its own RTG, the chip will be <em>fried</em>. Therefore, it is assumed that you will follow procedure and keep chips connected to their corresponding RTG when they're in the same room, and away from other RTGs otherwise.</p>
<p>These microchips sound very interesting and useful to your current activities, and you'd like to try to retrieve them.  The fourth floor of the facility has an assembling machine which can make a self-contained, shielded computer for you to take with you - that is, if you can bring it all of the RTGs and microchips.</p>
<p>Within the radiation-shielded part of the facility (in which it's safe to have these pre-assembly RTGs), there is an elevator that can move between the four floors. Its capacity rating means it can carry at most yourself and two RTGs or microchips in any combination. (They're rigged to some heavy diagnostic equipment - the assembling machine will detach it for you.) As a security measure, the elevator will only function if it contains at least one RTG or microchip. The elevator always stops on each floor to recharge, and this takes long enough that the items within it and the items on that floor can irradiate each other. (You can prevent this if a Microchip and its Generator end up on the same floor in this way, as they can be connected while the elevator is recharging.)</p>
<p>You make some notes of the locations of each component of interest (your puzzle input). Before you don a hazmat suit and start moving things around, you'd like to have an idea of what you need to do.</p>
<p>When you enter the containment area, you and the elevator will start on the first floor.</p>
<p>For example, suppose the isolated area has the following arrangement:</p>
<pre class="wrap"><code>The first floor contains a hydrogen-compatible microchip and a lithium-compatible microchip.
The second floor contains a hydrogen generator.
The third floor contains a lithium generator.
The fourth floor contains nothing relevant.
</code></pre>
<p>As a diagram (<code>F#</code> for a Floor number, <code>E</code> for Elevator, <code>H</code> for Hydrogen, <code>L</code> for Lithium, <code>M</code> for Microchip, and <code>G</code> for Generator), the initial state looks like this:</p>
<pre><code>F4 .  .  .  .  .  
F3 .  .  .  LG .  
F2 .  HG .  .  .  
F1 E  .  HM .  LM 
</code></pre>
<p>Then, to get everything up to the assembling machine on the fourth floor, the following steps could be taken:</p>
<ul>
<li><p>Bring the Hydrogen-compatible Microchip to the second floor, which is safe because it can get power from the Hydrogen Generator:</p><pre><code>F4 .  .  .  .  .  
F3 .  .  .  LG .  
F2 E  HG HM .  .  
F1 .  .  .  .  LM 
</code></pre></li>
<li><p>Bring both Hydrogen-related items to the third floor, which is safe because the Hydrogen-compatible microchip is getting power from its generator:</p><pre><code>F4 .  .  .  .  .  
F3 E  HG HM LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  LM 
</code></pre></li>
<li><p>Leave the Hydrogen Generator on floor three, but bring the Hydrogen-compatible Microchip back down with you so you can still use the elevator:</p><pre><code>F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 E  .  HM .  .  
F1 .  .  .  .  LM 
</code></pre></li>
<li><p>At the first floor, grab the Lithium-compatible Microchip, which is safe because Microchips don't affect each other:</p><pre><code>F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 .  .  .  .  .  
F1 E  .  HM .  LM 
</code></pre></li>
<li><p>Bring both Microchips up one floor, where there is nothing to fry them:</p><pre><code>F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 E  .  HM .  LM 
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Bring both Microchips up again to floor three, where they can be temporarily connected to their corresponding generators while the elevator recharges, preventing either of them from being fried:</p><pre><code>F4 .  .  .  .  .  
F3 E  HG HM LG LM 
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Bring both Microchips to the fourth floor:</p><pre><code>F4 E  .  HM .  LM 
F3 .  HG .  LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Leave the Lithium-compatible microchip on the fourth floor, but bring the Hydrogen-compatible one so you can still use the elevator; this is safe because although the Lithium Generator is on the destination floor, you can connect Hydrogen-compatible microchip to the Hydrogen Generator there:</p><pre><code>F4 .  .  .  .  LM 
F3 E  HG HM LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Bring both Generators up to the fourth floor, which is safe because you can connect the Lithium-compatible Microchip to the Lithium Generator upon arrival:</p><pre><code>F4 E  HG .  LG LM 
F3 .  .  HM .  .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Bring the Lithium Microchip with you to the third floor so you can use the elevator:</p><pre><code>F4 .  HG .  LG .  
F3 E  .  HM .  LM 
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
<li><p>Bring both Microchips to the fourth floor:</p><pre><code>F4 E  HG HM LG LM 
F3 .  .  .  .  .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
</code></pre></li>
</ul>
<p>In this arrangement, it takes <code>11</code> steps to collect all of the objects at the fourth floor for assembly. (Each elevator stop counts as one step, even if nothing is added to or removed from it.)</p>
<p>In your situation, what is the <em>minimum number of steps</em> required to bring all of the objects to the fourth floor?</p>
</article>


In [87]:
from collections import Counter, deque
from copy import deepcopy
from itertools import combinations
from more_itertools import transpose
from tabulate import tabulate


# Rules:
# -1-   An RTG powering a microchip is still dangerous to other microchips.
# -2-   A chip is ever left in the same area as another RTG, and
#       it's not connected to its own RTG, the chip will be fried
# -3-   Goal get them all on the 4th floor
# -4-   The elevator's capacity rating means it can carry at most yourself and
#       two RTGs or microchips in any combination.
# -5-   The elevator will only function if it contains at least one RTG or microchip.
# -6-   The elevator always stops on each floor
#
# fmt: off

ds = [
    (
        [
            # E     HG    HM    LG    LM      
            [".",  ".",  ".",  ".",  "."],
            [".",  ".",  ".", "LG",  "."],
            [".", "HG",  ".",  ".",  "."],
            ["E",  ".", "HM",  ".", "LM"],
        ],
        True, # VALID
    ),
    (
        [
            # E   HG    HM     LG   LM    
            [".",  ".",  ".",  ".",  "."],
            [".",  ".",  ".", "LG",  "."],
            ["E", "HG",  ".",  ".", "LM"],
            [".",  ".", "HM",  ".",  "."],
        ],
        False, # INVALID
    ),
    (
        [
            # E   HG    HM     LG   LM
            [".",  ".",  ".",  ".",  "."],
            [".",  ".",  ".", "LG",  "."],
            ["E", "HG", "HM",  ".",  "."],
            [".",  ".",  ".",  ".", "LM"],
        ],
        True, # VALID
    ),
    (
        [
            # E   HG    HM     LG   LM    
            [".",  ".",  ".",  ".",  "."],
            ["E", "HG", "HM", "LG",  "."],
            [".",  ".",  ".",  ".", "LM"],
            [".",  ".",  ".",  ".",  "."],
        ],
        True, # VALID
    ),
    (
        [
            # E   HG    HM     LG   LM    
            ["E", "HG", "HM", "LG", "LM"],
            [".",  ".",  ".",  ".",  "."],
            [".",  ".",  ".",  ".",  "."],
            [".",  ".",  ".",  ".",  "."],
        ],
        True, # VALID
    ),
    (
        [
        #   ["E",  "PLG", "PLM", "PRG", "PRM", "RUG", "RUM", "STG", "STM", "THG", "THM"]
            [".",    ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
            [".",    ".",   ".", "PRG", "PRM", "RUG", "RUM",   ".",   ".",   ".",   "."],
            [".",    ".", "PLM",   ".",   ".",   ".",   ".",   ".", "STM",   ".",   "."],
            ["E",  "PLG",   ".",   ".",   ".",   ".",   ".", "STG",   ".", "THG", "THM"],
        ],
        True, # Valid
    )
]
# fmt: on


def get_goal(d: list[list[str]]) -> list[str]:
    return [[s for s in r if s != "."][0] for r in transpose(d)]


assert get_goal(ds[0][0]) == ["E", "HG", "HM", "LG", "LM"]


def is_valid_row(row: list[str]) -> bool:
    gs = {o[:-1] for o in row if o[-1] == "G"}
    if not gs:
        return True
    ms = {o[:-1] for o in row if o[-1] == "M"}
    return not ms or not ms - gs


def is_valid_diagram(d: list[list[str]]) -> bool:
    return all(is_valid_row(row) for row in d)


assert all(is_valid_diagram(d) == e for d, e in ds)


name_to_column = {n: i for i, n in enumerate(get_goal(ds[0][0]))}


def move_col(name_to_column, e, delta, load, nxt):
    if load is not None:
        col = name_to_column[load]
        nxt[e][col] = "."
        nxt[e + delta][col] = load


self = deepcopy(ds[0][0])
move_col(name_to_column, 3, -1, "LM", self)
move_col(name_to_column, 2, 1, "LM", self)
assert self == ds[0][0]


def is_valid(left, right):
    return (
        left is None
        or right is None
        or left[-1] == right[-1]
        or left[:-1] == right[:-1]
    )


def loads(d, fr):
    return combinations([None] + [a for a in d[fr] if a != "." and a != "E"], 2)


assert all(is_valid(l, r) for l, r in loads(ds[0][0], 3))

assert Counter(is_valid(l, r) for l, r in loads(ds[4][0], 0)) == Counter(
    {True: 8, False: 2}
)
assert Counter(is_valid(l, r) for l, r in loads(ds[5][0], 3)) == Counter(
    {True: 8, False: 2}
)
assert Counter(is_valid(l, r) for l, r in loads(ds[5][0], 2)) == Counter(
    {True: 3, False: 0}
)
# The first floor contains
#   -1-   a hydrogen-compatible microchip and  (HM)
#   -2-   a lithium-compatible microchip.      (LM)
# The second floor contains
#   -3-   a hydrogen generator.                (HG)
# The third floor contains
#   -4-   a lithium generator.                 (LG)
# The fourth floor contains nothing relevant.

# fmt: off
diagram = [
    # E   HG    HM     LG   LM
    [".",  ".",  ".",  ".",  "."],
    [".",  ".",  ".", "LG",  "."],
    [".", "HG",  ".",  ".",  "."],
    ["E",  ".", "HM",  ".", "LM"],
]
# fmt: on


def minimum_steps(d: list[list[str]]) -> int:
    goal = get_goal(d)
    name_to_column = {n: i for i, n in enumerate(goal)}

    queue = deque([(len(d) - 1, d)])
    visited = set()

    step = 0

    while queue:
        queue = queue
        for _ in range(len(queue)):
            e, cur = queue.popleft()

            if cur[0] == goal:
                return step

            str_cur = str(cur)

            if str_cur in visited:
                continue

            visited.add(str_cur)

            # print(tabulate(cur, tablefmt="rounded_grid"))

            for delta in [-1, 1]:
                if not 0 <= e + delta < 4:
                    continue

                for comb in loads(cur, e):
                    left, right = comb
                    if not is_valid(left, right):
                        continue
                    nxt = deepcopy(cur)
                    move_col(name_to_column, e, delta, left, nxt)
                    move_col(name_to_column, e, delta, right, nxt)
                    move_col(name_to_column, e, delta, "E", nxt)
                    if is_valid_row(nxt[e]) and is_valid_row(nxt[e + delta]):
                        queue.append((e + delta, nxt))
        step += 1

    return inf


minimum_steps(diagram)

11

In [88]:
#  PART I
#
# The first floor contains
#   -1-   a thulium generator,                  THG
#   -2-   a thulium-compatible microchip,       THM
#   -3-   a plutonium generator, and            PLG
#   -4-   a strontium generator.                STG
# The second floor contains
#   -5-   a plutonium-compatible microchip and  PLM
#   -6-   a strontium-compatible microchip.     STM
# The third floor contains
#   -7-   a promethium generator,               PRG
#   -8-   a promethium-compatible microchip,    PRM
#   -9-   a ruthenium generator, and            RUG
#  -10-   a ruthenium-compatible microchip.     RUM
# The fourth floor contains nothing relevant.


# fmt: off
diagram_challenge = [
#   ["E",  "PLG", "PLM", "PRG", "PRM", "RUG", "RUM", "STG", "STM", "THG", "THM"]
    [".",    ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".",   ".", "PRG", "PRM", "RUG", "RUM",   ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".",   ".",   ".",   ".",   ".", "STM",   ".",   "."],
    ["E",  "PLG",   ".",   ".",   ".",   ".",   ".", "STG",   ".", "THG", "THM"],
]
# fmt: on


print(tabulate(diagram_challenge, tablefmt="rounded_grid"))
minimum_steps(diagram_challenge)  # too slow

╭───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────╮
│ . │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ . │ .   │ .   │ PRG │ PRM │ RUG │ RUM │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ . │ .   │ PLM │ .   │ .   │ .   │ .   │ .   │ STM │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E │ PLG │ .   │ .   │ .   │ .   │ .   │ STG │ .   │ THG │ THM │
╰───┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────╯


31

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>31</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You step into the cleanroom separating the lobby from the isolated area and put on the hazmat suit.</p>
<p>Upon entering the isolated containment area, however, you notice some extra parts on the first floor that weren't listed on the record outside:</p>
<ul>
<li>An elerium generator.</li>
<li>An elerium-compatible microchip.</li>
<li>A dilithium generator.</li>
<li>A dilithium-compatible microchip.</li>
</ul>
<p>These work just like the other generators and microchips. You'll have to get them up to assembly as well.</p>
<p>What is the <em>minimum number of steps</em> required to bring all of the objects, including these four new ones, to the fourth floor?</p>
</article>

</main>


a+b=60 |\_ 40 + b + b = 60 => 2b=60-40=> b=10 => a + 10 = 60 => a = 50
a-b=40 => a = 40+b |
a/b=x

5-/10=5


In [89]:
# fmt: off
diagram_challenge = [
#   ["E",  "PLG", "PLM", "PRG", "PRM", "RUG", "RUM", "STG", "STM", "THG", "THM", "ELG", "ELM", "DLG", "DLM"]
    [".",    ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".",   ".", "PRG", "PRM", "RUG", "RUM",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".",   ".",   ".",   ".",   ".", "STM",   ".",   ".",   ".",   ".",   ".",   "."],
    ["E",  "PLG",   ".",   ".",   ".",   ".",   ".", "STG",   ".", "THG", "THM", "ELG", "ELM", "DLG", "DLM"],
]
# fmt: on
print(tabulate(diagram_challenge, tablefmt="rounded_grid"))
# minimum_steps(diagram_challenge)  # too slow
# 257 too big

╭───┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────╮
│ . │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ . │ .   │ .   │ PRG │ PRM │ RUG │ RUM │ .   │ .   │ .   │ .   │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ . │ .   │ PLM │ .   │ .   │ .   │ .   │ .   │ STM │ .   │ .   │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤
│ E │ PLG │ .   │ .   │ .   │ .   │ .   │ STG │ .   │ THG │ THM │ ELG │ ELM │ DLG │ DLM │
╰───┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────╯


KeyboardInterrupt: 

In [ ]:
print("Example:", sum(2 * sum([2, 1, 1, 0][:x]) - 3 for x in range(1, 4)))
print("Part I :", sum(2 * sum([4, 2, 4, 0][:x]) - 3 for x in range(1, 4)))
print("Part II:", sum(2 * sum([8, 2, 4, 0][:x]) - 3 for x in range(1, 4)))

Example: 9
Part I : 31
Part II: 55


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>55</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="11/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Radioisotope+Thermoelectric+Generators%22+%2D+Day+11+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F11&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Radioisotope+Thermoelectric+Generators%22+%2D+Day+11+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F11';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>


In [90]:
# Any complete pairs on floor 1 add 12 steps to the solution

# Complete pairs on floor 2 add 8 steps

# Complete pairs on floor 3 add 4 steps

# So you can remove all such pairs from the initial question,
# then solve the remaining puzzle however you like (I did it by hand).
# This realization makes part 2 SUPER easy.

# fmt: off
diagram_challenge_I = [
#   ["E",  "PLG", "PLM", "PRG", "PRM", "RUG", "RUM", "STG", "STM", "THG", "THM"]
    [".",    ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".",   ".", "PRG", "PRM", "RUG", "RUM",   ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".",   ".",   ".",   ".",   ".", "STM",   ".",   "."],
    ["E",  "PLG",   ".",   ".",   ".",   ".",   ".", "STG",   ".", "THG", "THM"],
]

extra_steps = 12 + 2 * 4

reduced_challenge_I = [
    [".",    ".",   ".",   ".",   "."],
    [".",    ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".", "STM"],
    ["E",  "PLG",   ".", "STG",   "."],
]

# fmt: on


print(tabulate(reduced_challenge_I, tablefmt="rounded_grid"))
minimum_steps(reduced_challenge_I) + extra_steps

╭───┬─────┬─────┬─────┬─────╮
│ . │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┤
│ . │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┤
│ . │ .   │ PLM │ .   │ STM │
├───┼─────┼─────┼─────┼─────┤
│ E │ PLG │ .   │ STG │ .   │
╰───┴─────┴─────┴─────┴─────╯


31

In [91]:
# Any complete pairs on floor 1 add 12 steps to the solution

# Complete pairs on floor 2 add 8 steps

# Complete pairs on floor 3 add 4 steps

# So you can remove all such pairs from the initial question,
# then solve the remaining puzzle however you like (I did it by hand).
# This realization makes part 2 SUPER easy.

# fmt: off
diagram_challenge_I = diagram_challenge = [
#   ["E",  "PLG", "PLM", "PRG", "PRM", "RUG", "RUM", "STG", "STM", "THG", "THM", "ELG", "ELM", "DLG", "DLM"]
    [".",    ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".",   ".", "PRG", "PRM", "RUG", "RUM",   ".",   ".",   ".",   ".",   ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".",   ".",   ".",   ".",   ".", "STM",   ".",   ".",   ".",   ".",   ".",   "."],
    ["E",  "PLG",   ".",   ".",   ".",   ".",   ".", "STG",   ".", "THG", "THM", "ELG", "ELM", "DLG", "DLM"],
]

extra_steps = 3 * 12 + 2 * 4

reduced_challenge_I = [
    [".",    ".",   ".",   ".",   "."],
    [".",    ".",   ".",   ".",   "."],
    [".",    ".", "PLM",   ".", "STM"],
    ["E",  "PLG",   ".", "STG",   "."],
]

# fmt: on


print(tabulate(reduced_challenge_I, tablefmt="rounded_grid"))
minimum_steps(reduced_challenge_I) + extra_steps

╭───┬─────┬─────┬─────┬─────╮
│ . │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┤
│ . │ .   │ .   │ .   │ .   │
├───┼─────┼─────┼─────┼─────┤
│ . │ .   │ PLM │ .   │ STM │
├───┼─────┼─────┼─────┼─────┤
│ E │ PLG │ .   │ STG │ .   │
╰───┴─────┴─────┴─────┴─────╯


55